# AWS Python SDK(boto3) DataZone Client의 Search 메서드를 이용하여 Asset 메타데이터 저장

- 아래 **현재 실행 중인 IAM 정보**는 실제 호출할 주체를 확인하기 위해 실행



In [1]:
import boto3

sts = boto3.client("sts")
identity = sts.get_caller_identity()

print("✅ 현재 실행 중인 IAM Identity 정보:")
print(f"Account ID : {identity['Account']}")
print(f"UserId     : {identity['UserId']}")
print(f"Arn        : {identity['Arn']}")


✅ 현재 실행 중인 IAM Identity 정보:
Account ID : 533616270150
UserId     : AIDAXYPQCDNDD6RMCGL25
Arn        : arn:aws:iam::533616270150:user/mj.kwon@sk.com


### Asset 메타데이터 전처리 과정을 **SMUS Notebook** 내에서 실행할 경우에는 아래와 같이 출력됨.

✅ **현재 실행 중인 IAM Identity 정보**

| 항목 | 값 |
|------|----|
| Account ID | `533616270150` |
| UserId | `AROAXYPQCDNDMUFTA7JFO:SageMaker` |
| Arn | `arn:aws:sts::533616270150:assumed-role/datazone_usr_role_bknisgxzopjuhk_4oa8qs5krhk8ig/SageMaker` |

---

### SMUS Notebook에서 실행할 경우 아래 정책을 **`datazone_usr_role_bknisgxzopjuhk_4oa8qs5krhk8ig`** 역할에 추가해야 함:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowDataZoneReadAccess",
      "Effect": "Allow",
      "Action": [
        "datazone:Search",
        "datazone:GetAsset",
        "datazone:GetDomain",
        "datazone:ListProjects",
        "datazone:GetProject"
      ],
      "Resource": [
        "arn:aws:datazone:ap-northeast-2:533616270150:domain/dzd-cjvglgj4d43fmg",
        "arn:aws:datazone:ap-northeast-2:533616270150:domain/dzd-cjvglgj4d43fmg/*"
      ]
    }
  ]
}


### SMUS Domain 내 전체 Asset을 조회
- AWS SDK DataZone Client의 Search메서드를 사용하며, 아래 자세한 정보 확인 가능
- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/datazone/client/search.html
- Search메서드는 searchScope 기준으로 조회가 가능하며, 아래와 같이 4가지를 지원
- searchScope: "ASSET" || "GLOSSARY" || "GLOSSARY_TERM" || "DATA_PRODUCT"

### 아래의 코드는 전체 Asset 중 GlueTableAssetType + listing_status == "ACTIVE" (PUBLISHED)만 찾아서 해당 Asset 별 메타데이터를 "[Asset명]_[AssetID].json"형식으로 저장하는 예시 

In [13]:
import boto3
import json
import os
import re
from concurrent.futures import ThreadPoolExecutor 
from datetime import datetime, timezone 

# --- 환경 설정 ---
region = "ap-northeast-2"
dz = boto3.client("datazone", region_name=region)
domain_id = "dzd-3ojk7mnm02q5lk"
MAX_THREADS = 50 

# 결과 저장 폴더 (1회성 전체 추출용)
output_dir = "sample_result_data/glue_assets_full"
os.makedirs(output_dir, exist_ok=True)


# ----------------------------------------------------------------------
# 💡 함수 정의: get_asset 호출 및 상태 필터링 (병렬 처리 대상)
# ----------------------------------------------------------------------

def fetch_and_filter_asset(asset_info):
    """개별 에셋의 상세 정보를 가져와 게시 상태가 ACTIVE인지 확인합니다."""
    asset_id = asset_info["id"]
    try:
        asset_resp = dz.get_asset(
            domainIdentifier=domain_id,
            identifier=asset_id
        )
        
        # 'listing' 블록에서 'listingStatus' 확인 (ACTIVE = PUBLISHED)
        listing_block = asset_resp.get("listing", {}) or {}
        listing_status = listing_block.get("listingStatus", "UNKNOWN")
        
        if listing_status == "ACTIVE":
            asset_info.update({"metadata": asset_resp})
            return asset_info
            
        return None 

    except Exception as e:
        print(f"❌ Error fetching asset {asset_id} in thread: {e}")
        return None

# ----------------------------------------------------------------------
# 2️⃣ 모든 프로젝트별 GlueTableAssetType의 ID 수집 (필터링 없음)
# ----------------------------------------------------------------------
projects_resp = dz.list_projects(domainIdentifier=domain_id)
projects = {p["id"]: p["name"] for p in projects_resp.get("items", [])}
all_glue_asset_ids = []

print("\n--- Step 2: Collecting ALL Glue Asset IDs (No Filter) ---")

for project_id, project_name in projects.items():
    print(f"📁 Searching ALL assets in project: {project_name}")

    next_token = None
    while True:
        # 델타 필터링 파라미터(filters)를 넣지 않습니다.
        params = {
            "domainIdentifier": domain_id,
            "owningProjectIdentifier": project_id,
            "searchScope": "ASSET",
        }
        if next_token:
            params["nextToken"] = next_token

        resp = dz.search(**params)

        for item in resp.get("items", []):
            asset_item = item.get("assetItem", {})
            asset_type_id = asset_item.get("typeIdentifier")
            
            if asset_type_id == "amazon.datazone.GlueTableAssetType":
                all_glue_asset_ids.append({
                    "id": asset_item.get("identifier"),
                    "name": asset_item.get("name"),
                    "project_id": project_id,
                    "project_name": project_name,
                })
        next_token = resp.get("nextToken")
        if not next_token:
            break

print(f"\n✅ Total {len(all_glue_asset_ids)} GlueTableAssetType assets found.")

# ----------------------------------------------------------------------
# 3️⃣ get_asset 호출 및 최종 필터링 (병렬 처리 실행)
# ----------------------------------------------------------------------

published_glue_assets = []

print(f"\n--- Step 3: Fetching details using {MAX_THREADS} threads ---")

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    results = executor.map(fetch_and_filter_asset, all_glue_asset_ids)
    
    for result in results:
        if result:
            published_glue_assets.append(result)
            # print(f"✔ Final ACTIVE Asset: {result.get('name')}")

print(f"\n✅ Final Count: {len(published_glue_assets)} PUBLISHED assets.")

# ----------------------------------------------------------------------
# 4️⃣ JSON 저장 (전체 파일을 저장)
# ----------------------------------------------------------------------
print("\n--- Step 4: Saving ALL JSON files ---")
now_utc = datetime.now(timezone.utc)

for asset in published_glue_assets:
    asset_id = asset["id"]
    asset_name = asset["name"]
    project_name = asset["project_name"]
    metadata = asset["metadata"]

    safe_asset_name = re.sub(r'[^a-zA-Z0-9가-힣_\-]', '_', asset_name)
    safe_project_name = re.sub(r'[^a-zA-Z0-9가-힣_\-]', '_', project_name)
    output_path = os.path.join(output_dir, f"{safe_asset_name}_{safe_project_name}.json") 

    try:
        schema = metadata.get("schema") or metadata.get("glueTable", {}).get("columns")

        asset_data = {
            "identifier": asset_id,
            "name": asset_name,
            "project": asset["project_name"],
            "typeIdentifier": metadata.get("typeIdentifier"),
            "metadata": metadata,
            "schema": schema,
            "extraction_time": now_utc.isoformat() 
        }

        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(asset_data, f, default=str, ensure_ascii=False, indent=2)

        print(f"✅ Saved Asset: {output_path}")

    except Exception as e:
        print(f"❌ Failed to process and save asset {asset_name} ({asset_id}): {e}")

print("\n🎉 Full asset metadata extraction complete!")


--- Step 2: Collecting ALL Glue Asset IDs (No Filter) ---
📁 Searching ALL assets in project: GenerativeAIModelGovernanceProject
📁 Searching ALL assets in project: admin-project-533616270150
📁 Searching ALL assets in project: My_Project_mgu2iq14

✅ Total 50 GlueTableAssetType assets found.

--- Step 3: Fetching details using 50 threads ---

✅ Final Count: 3 PUBLISHED assets.

--- Step 4: Saving ALL JSON files ---
✅ Saved Asset: sample_result_data/glue_assets_full/s3_trail_events_admin-project-533616270150.json
✅ Saved Asset: sample_result_data/glue_assets_full/store_details_admin-project-533616270150.json
✅ Saved Asset: sample_result_data/glue_assets_full/store_details_My_Project_mgu2iq14.json

🎉 Full asset metadata extraction complete!


## 신규, 수정, 삭제 판단하여 asset 목록 관리

In [27]:
import boto3
import json
import os
import re
from concurrent.futures import ThreadPoolExecutor 
from datetime import datetime, timedelta, timezone 

# --- 환경 설정 ---
region = "ap-northeast-2"
dz = boto3.client("datazone", region_name=region)
domain_id = "dzd-3ojk7mnm02q5lk"
MAX_THREADS = 50 

# 💡 이 폴더에 저장/삭제/덮어쓰기가 발생하며, KB Data Source가 이 폴더를 바라봐야 합니다.
KB_S3_SYNC_DIR = "sample_result_data/glue_assets_kb_sync" 
os.makedirs(KB_S3_SYNC_DIR, exist_ok=True)
PUBLISHED_ASSET_IDS_FILE = "published_asset_ids_snapshot.json" # ID 스냅샷 저장 파일

# ----------------------------------------------------------------------
# 💡 Step 1: 시간 계산 및 필터 설정
# ----------------------------------------------------------------------
now_utc = datetime.now(timezone.utc)
# 어제 날짜 (24시간 전)의 UTC 시각을 기준점으로 설정하여 신규 생성된 에셋을 찾습니다.
delta_start_time = now_utc - timedelta(days=1) 
delta_start_iso = delta_start_time.isoformat().replace('+00:00', 'Z') 

print(f"⏰ Delta window start time for NEW assets: {delta_start_iso}")

# 필터 구조: createdAt이 델타 기간 이후인 에셋만 1차 검색 (신규 생성 감지)
creation_filter_final = {
    "and": [
        {
            "filter": { 
                "attribute": "createdAt",
                "value": f">={delta_start_iso}" # 비표준 문자열 필터 사용
            }
        }
    ]
}

# ----------------------------------------------------------------------
# 💡 Step 0: 이전 실행 시점의 PUBLISHED Asset ID 목록을 로드
# ----------------------------------------------------------------------
previous_published_ids = set()
if os.path.exists(PUBLISHED_ASSET_IDS_FILE):
    with open(PUBLISHED_ASSET_IDS_FILE, 'r') as f:
        previous_published_ids = set(json.load(f))
    print(f"Loaded {len(previous_published_ids)} IDs from previous run.")


# ----------------------------------------------------------------------
# 💡 Step 3 함수 정의 (상태 확인 및 메타데이터 추출)
# ----------------------------------------------------------------------
def fetch_and_filter_asset(asset_info):
    """개별 에셋의 상세 정보를 가져와 상태가 ACTIVE인지 확인하고 메타데이터를 반환합니다."""
    # (함수 로직은 이전과 동일하며, 정확성 유지를 위해 재정의하지 않습니다.)
    asset_id = asset_info["id"]
    try:
        asset_resp = dz.get_asset(domainIdentifier=domain_id, identifier=asset_id)
        listing_status = (asset_resp.get("listing", {}) or {}).get("listingStatus", "UNKNOWN")
        
        if listing_status == "ACTIVE":
            asset_info.update({"metadata": asset_resp})
            print(asset_resp)
            return asset_info
        return None 
    except Exception as e:
        # Asset이 완전히 삭제된 경우 get_asset에서 예외 발생 가능
        if "ResourceNotFoundException" in str(e):
            return {"id": asset_id, "status": "DELETED"}
        return None

# ----------------------------------------------------------------------
# 2️⃣ 모든 GlueTableAssetType의 ID 수집 (전체 + 신규 생성)
# ----------------------------------------------------------------------
projects_resp = dz.list_projects(domainIdentifier=domain_id)
projects = {p["id"]: p["name"] for p in projects_resp.get("items", [])}
all_glue_asset_ids = []

for project_id in projects.keys():
    # 1. 신규 생성된 에셋 ID 수집 (creation_filter_final 사용)
    # 2. 모든 에셋 ID를 수집 (lastUpdatedAt 필터가 없으므로)
    
    # 💡 최적화를 위해, 현재는 모든 에셋을 가져옵니다. (lastUpdatedAt 필터링이 안되므로)
    # 만약 에셋 수가 수천 개가 아니라면 이 방법이 가장 안전합니다.
    # 만약 수만 개 이상이라면, 전체 검색 시간을 줄일 방법이 없습니다.

    next_token = None
    while True:
        params = {
            "domainIdentifier": domain_id,
            "owningProjectIdentifier": project_id,
            "searchScope": "ASSET",
            # 필터를 사용하지 않고 모든 에셋을 가져와서 로직에서 덮어씁니다.
        }
        if next_token:
            params["nextToken"] = next_token

        resp = dz.search(**params)
        
        for item in resp.get("items", []):
            asset_item = item.get("assetItem", {})
            if asset_item.get("typeIdentifier") == "amazon.datazone.GlueTableAssetType":
                all_glue_asset_ids.append({
                    "id": asset_item.get("identifier"),
                    "name": asset_item.get("name"),
                    "project_id": project_id,
                })
        next_token = resp.get("nextToken")
        if not next_token:
            break

# ----------------------------------------------------------------------
# 3️⃣ get_asset 호출 및 최종 필터링 (병렬 처리 실행)
# ----------------------------------------------------------------------
current_published_ids = set()
assets_to_save = []
deleted_assets = []
MAX_THREADS = 50 

print(f"\n--- Step 3: Fetching details using {MAX_THREADS} threads ---")

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    results = executor.map(fetch_and_filter_asset, all_glue_asset_ids)
    
    for result in results:
        if result:
            if result.get("status") == "DELETED":
                deleted_assets.append(result)
            elif "metadata" in result:
                assets_to_save.append(result)
                current_published_ids.add(result["id"])

print(f"Found {len(assets_to_save)} ACTIVE assets.")

# ----------------------------------------------------------------------
# 4️⃣ S3/KB 폴더 동기화 로직 (3가지 시나리오 처리)
# ----------------------------------------------------------------------

# 💡 4-A: PUBLISHED 해제된 Asset 감지 및 삭제 (시나리오 3)
# 이전 ID 목록에는 있었지만 현재 ID 목록에는 없는 Asset
ids_to_delete = previous_published_ids - current_published_ids

for asset_id in ids_to_delete:
    # KB S3 경로에서 해당 파일 이름 패턴을 찾아 삭제해야 합니다.
    # 정확한 파일명을 알 수 없으므로, ID가 포함된 파일을 검색해야 하지만
    # 여기서는 파일명 규칙을 기반으로 임시 삭제를 시뮬레이션합니다.
    
    # 실제 구현 시에는 S3 API를 사용하여 해당 Asset ID를 포함하는 파일을 검색 후 삭제해야 합니다.
    print(f"🗑️ Deleting file for UNPUBLISHED/DELETED Asset ID: {asset_id} (KB Sync)")
    # os.remove(os.path.join(KB_S3_SYNC_DIR, f"*_{asset_id}.json")) # 실제 S3 삭제 로직 필요

# 💡 4-B: 신규 생성 및 변경된 Asset 저장 (시나리오 1 & 2)
# KB Sync는 파일 생성, 수정(덮어쓰기)을 모두 감지합니다.
for asset in assets_to_save:
    asset_id = asset["id"]
    asset_name = asset["name"]
    metadata = asset["metadata"]

    # 파일명은 ID와 이름으로 고정하여 KB Sync가 추적할 수 있도록 합니다.
    safe_name = re.sub(r'[^a-zA-Z0-9가-힣_\-]', '_', asset_name)
    output_path = os.path.join(KB_S3_SYNC_DIR, f"{safe_name}_{asset_id}.json") 

    # 덮어쓰기 로직: KB Sync는 파일 수정으로 인식 (시나리오 1 & 2 모두 처리)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(
            {"identifier": asset_id, "name": asset_name, "metadata": metadata, "extraction_time": now_utc.isoformat()}, 
            f, default=str, ensure_ascii=False, indent=2
        )
    
    action = "CREATED/UPDATED"
    if asset_id in previous_published_ids:
        action = "UPDATED" # 이전에 존재했으면 업데이트로 간주
    else:
        action = "CREATED" # 이전에 없었으면 신규 생성으로 간주

    print(f"✅ KB File {action}: {output_path}")

# 💡 4-C: 현재 PUBLISHED ID 목록 저장 (다음 실행을 위해)
with open(PUBLISHED_ASSET_IDS_FILE, "w") as f:
    json.dump(list(current_published_ids), f)

print("\n🎉 Daily Delta Asset Management Complete.")

⏰ Delta window start time for NEW assets: 2025-11-19T01:21:14.385217Z

--- Step 3: Fetching details using 50 threads ---
{'ResponseMetadata': {'RequestId': 'cb2edae0-30e1-4b01-b871-7098618f63dc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/json', 'content-length': '3619', 'connection': 'keep-alive', 'date': 'Thu, 20 Nov 2025 01:21:17 GMT', 'x-amzn-trace-id': 'Root=1-691e6d0d-4d2d1b0816e36c29040d3957', 'x-amzn-requestid': 'cb2edae0-30e1-4b01-b871-7098618f63dc', 'x-amz-apigw-id': 'UUX6GFQJoE0ESSA=', 'x-cache': 'Miss from cloudfront', 'via': '1.1 f8928b044dc72c905cb19cef767da0aa.cloudfront.net (CloudFront)', 'x-amz-cf-pop': 'ICN57-P1', 'x-amz-cf-id': 'QXp9s5rrqJMwvXfZwsL4bu_dkX1JO8ddQNuBhgZ3yA9J9wFk4dm_0w=='}, 'RetryAttempts': 0}, 'createdAt': datetime.datetime(2025, 10, 17, 8, 27, 3, 260000, tzinfo=tzlocal()), 'createdBy': 'SYSTEM', 'domainId': 'dzd-3ojk7mnm02q5lk', 'externalIdentifier': 'arn:aws:glue:ap-northeast-2:533616270150:table/cloudtrail_logs/s3_trail_even